In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from rtb.datasets import get_dataset
from rtb.utils import rolling_window_sampler, one_window_sampler

In [25]:
ds = get_dataset(
    name="rtb-product",
    root="../../data",
    # default is books-5-core
    # category="fashion",
    # use_5_core=False,
    process=True,
)

processing db...
reading product info from ../../data/rtb-product/books-5-core/raw/meta_Books.json...
done in 1.25 seconds.
converting to pandas dataframe...
done in 5.64 seconds.
processing product info...
done in 4.11 seconds.
reading review and customer info from ../../data/rtb-product/books-5-core/raw/Books_5.json...
done in 8.65 seconds.
converting to pandas dataframe...
done in 40.98 seconds.
processing review and customer info...
done in 1.64 seconds.
keeping only products common to product and review tables...
done in 9.77 seconds.
extracting customer table...
done in 8.41 seconds.
processing db took 80.87 seconds.
reindexing pkeys and fkeys...
reindexing pkeys and fkeys took 12.32 seconds.
saving table product...
done in 2.21 seconds.
saving table customer...
done in 0.38 seconds.
saving table review...
done in 60.04 seconds.


In [26]:
db = ds._db

In [27]:
customer = db.tables["customer"].df
product = db.tables["product"].df
review = db.tables["review"].df

In [28]:
db.tables["customer"].df

,customer_id,customer_name
0,0,TW Ervin II
1,1,Amazon Customer
2,2,Tekla Borner
3,3,Deborah K Woroniecki
4,4,E
...,...,...
27156207,1850188,Amazon Customer
27156208,1850189,AmizoP
27161776,1850190,JuicyBeans
27161778,1850191,Halley Cortez


In [29]:
product

,product_id,category,brand,title,description,price
16,0,"[Books, Literature & Fiction, Dramas & Plays]",Visit Amazon's William Shakespeare Page,Love's Labour's Lost: Performed by Derek Jacob...,William Shakespeare is widely regarded as the ...,20.93
25,1,"[Books, Children's Books, Literature & Fiction]",Louis (ed) UNTERMEYER,The golden treasury of poetry,Collection of Poetry,23.98
33,2,"[Books, Mystery, Thriller & Suspense]",Visit Amazon's Agatha Christie Page,The Blue Geranium (Miss Marple),None,8.82
42,3,"[Books, Humor & Entertainment]",Visit Amazon's Tom Baker Page,Who on Earth Is Tom Baker?,Tom Baker's autobiography covers his childhood...,159.95
61,4,"[Books, Christian Books & Bibles, Bible Study ...",Visit Amazon's Clarence Larkin Page,The Greatest Book on &quot;Dispensational Trut...,The Greatest Book on Dispensational Truth in t...,44.76
...,...,...,...,...,...,...
2933518,506007,None,"Tricia & Putman, Cara Goyer",Mason Jar Mayhem,"Spring has come to Sugarcreek, and the communi...",15.05
2933605,506008,None,Author,The Little Book of Pussy (2013-06-16) [Paperback],From the time The Big Penis Book was published...,17.0
2934395,506009,None,,The Age of Not Believing: A Christmas Tale by ...,None,42.93
2934407,506010,None,,The Charlies Angels' Casebook,None,17.87


In [30]:
review

,review_time,customer_id,product_id,rating,verified,review_text,summary
0,2005-03-30,0,14,5.0,False,"The King, the Mice and the Cheese by Nancy Gur...",A story children will love and learn from
1,2016-06-20,1,14,5.0,True,The kids loved it!,Five Stars
2,2016-01-24,2,14,5.0,True,My students (3 & 4 year olds) loved this book!...,Five Stars
3,2015-07-09,3,14,5.0,False,LOVE IT,Five Stars
4,2015-01-18,4,14,5.0,True,Great!,Five Stars
...,...,...,...,...,...,...,...
27164624,2012-12-02,319196,506010,5.0,False,"This past year fans of ""Charlie's Angels"" were...","""NOW THIS IS THE COMPANION TO THE ""CHARLIE'S A..."
27164625,2008-12-15,1407251,506010,5.0,True,"For any fan of Charlie's Angels, this book is ...",An Excellent Case!
27164626,2007-06-15,1373107,506010,3.0,False,To review this book I will divide it into two ...,Great Stories But A Little One-Sided
27164627,2018-05-27,319324,506011,3.0,True,"Great pictures, good recommendations but not t...",Don't buy if your looking for details and sett...


## LTV

In [31]:
window_size = ds.tasks["ltv"].window_sizes[0]
window_size

Timedelta('364 days 00:00:00')

### train table

In [32]:
# default time window sampler used inside make_train_table
time_window_df = rolling_window_sampler(
    ds.min_time, ds.train_max_time, window_size, window_size
)
time_window_df

,window_min_time,window_max_time
0,1996-06-25 00:00:01,1997-06-24 00:00:01
1,1997-06-24 00:00:01,1998-06-23 00:00:01
2,1998-06-23 00:00:01,1999-06-22 00:00:01
3,1999-06-22 00:00:01,2000-06-20 00:00:01
4,2000-06-20 00:00:01,2001-06-19 00:00:01
5,2001-06-19 00:00:01,2002-06-18 00:00:01
6,2002-06-18 00:00:01,2003-06-17 00:00:01
7,2003-06-17 00:00:01,2004-06-15 00:00:01
8,2004-06-15 00:00:01,2005-06-14 00:00:01
9,2005-06-14 00:00:01,2006-06-13 00:00:01


In [33]:
train_table = ds.make_train_table("ltv", window_size)

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

In [34]:
# here count is the number of products which contributed to the sum aggregation in ltv
train_table.df

,window_min_time,window_max_time,customer_id,ltv,count
0,1996-06-25 00:00:01,1997-06-24 00:00:01,1843200,0.00,0
1,1996-06-25 00:00:01,1997-06-24 00:00:01,1843201,0.00,0
2,1996-06-25 00:00:01,1997-06-24 00:00:01,1843203,0.00,0
3,1996-06-25 00:00:01,1997-06-24 00:00:01,1843205,0.00,0
4,1996-06-25 00:00:01,1997-06-24 00:00:01,1843206,0.00,0
...,...,...,...,...,...
31453276,2012-06-05 00:00:01,2013-06-04 00:00:01,1023989,24.09,2
31453277,2012-06-05 00:00:01,2013-06-04 00:00:01,1023991,29.24,2
31453278,2012-06-05 00:00:01,2013-06-04 00:00:01,1021954,0.00,0
31453279,2012-06-05 00:00:01,2013-06-04 00:00:01,1023588,433.85,36


In [35]:
# frequency of counts in percent
train_table.df["count"].value_counts(normalize=True) * 100

count
0      95.140863
1       2.509846
2       0.924326
3       0.471588
4       0.286008
         ...    
220     0.000003
295     0.000003
288     0.000003
603     0.000003
339     0.000003
Name: proportion, Length: 333, dtype: float64

_Note: this is very skewed! Test table is not this skewed (95% 0-counts vs 47%)._

## val table

In [36]:
# time sampler used inside make_val_table
time_window_df = one_window_sampler(ds.val_max_time, window_size)
time_window_df

,window_min_time,window_max_time
0,2016-07-07 00:00:01,2017-07-06


In [37]:
val_table = ds.make_val_table("ltv", window_size)

In [38]:
val_table.df

,window_min_time,window_max_time,customer_id,ltv,count
0,2014-04-16 00:00:01,2015-04-15,1843200,0.0,0
1,2014-04-16 00:00:01,2015-04-15,1843201,0.0,0
2,2014-04-16 00:00:01,2015-04-15,1843202,0.0,0
3,2014-04-16 00:00:01,2015-04-15,1843203,0.0,0
4,2014-04-16 00:00:01,2015-04-15,1843204,0.0,0
...,...,...,...,...,...
1850188,2014-04-16 00:00:01,2015-04-15,1841723,0.0,0
1850189,2014-04-16 00:00:01,2015-04-15,1842013,0.0,0
1850190,2014-04-16 00:00:01,2015-04-15,1842771,0.0,0
1850191,2014-04-16 00:00:01,2015-04-15,1843016,0.0,0


In [39]:
val_table.df["count"].value_counts(normalize=True) * 100

count
0       47.376679
1       17.258091
2       11.013446
3        7.313778
4        4.936566
          ...    
1171     0.000054
232      0.000054
228      0.000054
284      0.000054
240      0.000054
Name: proportion, Length: 302, dtype: float64

## Churn

In [40]:
window_size = ds.tasks["churn"].window_sizes[0]
window_size

Timedelta('364 days 00:00:00')

### train table

In [41]:
# default time window sampler used inside make_train_table
time_window_df = rolling_window_sampler(
    ds.min_time, ds.train_max_time, window_size, window_size
)
time_window_df

,window_min_time,window_max_time
0,1996-06-25 00:00:01,1997-06-24 00:00:01
1,1997-06-24 00:00:01,1998-06-23 00:00:01
2,1998-06-23 00:00:01,1999-06-22 00:00:01
3,1999-06-22 00:00:01,2000-06-20 00:00:01
4,2000-06-20 00:00:01,2001-06-19 00:00:01
5,2001-06-19 00:00:01,2002-06-18 00:00:01
6,2002-06-18 00:00:01,2003-06-17 00:00:01
7,2003-06-17 00:00:01,2004-06-15 00:00:01
8,2004-06-15 00:00:01,2005-06-14 00:00:01
9,2005-06-14 00:00:01,2006-06-13 00:00:01


In [43]:
train_table = ds.make_train_table("churn", window_size)

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

In [44]:
# here count is the number of products which contributed to the sum aggregation in ltv
train_table.df

,window_min_time,window_max_time,customer_id,churn
0,1996-06-25 00:00:01,1997-06-24 00:00:01,1843200,True
1,1996-06-25 00:00:01,1997-06-24 00:00:01,1843201,True
2,1996-06-25 00:00:01,1997-06-24 00:00:01,1843202,True
3,1996-06-25 00:00:01,1997-06-24 00:00:01,1843203,True
4,1996-06-25 00:00:01,1997-06-24 00:00:01,1843204,True
...,...,...,...,...
31453276,2012-06-05 00:00:01,2013-06-04 00:00:01,921595,False
31453277,2012-06-05 00:00:01,2013-06-04 00:00:01,921596,False
31453278,2012-06-05 00:00:01,2013-06-04 00:00:01,921597,True
31453279,2012-06-05 00:00:01,2013-06-04 00:00:01,921598,True


In [45]:
# frequency of churn in percent
train_table.df["churn"].value_counts(normalize=True) * 100

churn
True     95.140863
False     4.859137
Name: proportion, dtype: float64

_Note: this is very skewed! Val table is not this skewed (95% 0-counts vs 52%)._

## val table

In [46]:
# time sampler used inside make_val_table
time_window_df = one_window_sampler(ds.train_max_time, window_size)
time_window_df

,window_min_time,window_max_time
0,2014-04-16 00:00:01,2015-04-15


In [47]:
val_table = ds.make_val_table("churn", window_size)

In [48]:
val_table.df

,window_min_time,window_max_time,customer_id,churn
0,2014-04-16 00:00:01,2015-04-15,716800,False
1,2014-04-16 00:00:01,2015-04-15,716801,False
2,2014-04-16 00:00:01,2015-04-15,716802,True
3,2014-04-16 00:00:01,2015-04-15,716803,True
4,2014-04-16 00:00:01,2015-04-15,716804,True
...,...,...,...,...
1850188,2014-04-16 00:00:01,2015-04-15,1331195,True
1850189,2014-04-16 00:00:01,2015-04-15,1331196,True
1850190,2014-04-16 00:00:01,2015-04-15,1331197,True
1850191,2014-04-16 00:00:01,2015-04-15,1331198,False


In [49]:
val_table.df["churn"].value_counts(normalize=True) * 100

churn
False    52.623321
True     47.376679
Name: proportion, dtype: float64